# Extraction and classification of classical references from the ConDÉ corpus

Script written by Morgane Pica for a submission to the symposium ["Lire les classiques en Normandie"](https://rmblf.be/2022/02/04/appel-a-contribution-lire-les-classiques-en-normandie/) (oct 2022), to be written by herself and Mathieu Goux.

## Imports & declarations

In [1]:
from tqdm.notebook import tqdm #tqdm est bibliothèque qui permet d'avoir une barre de progression
import xml.etree.ElementTree as ET

ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
ET.register_namespace('xml','http://www.w3.org/XML/1998/namespace')

# Not all witnesses were enriched with reference identification.
witnesses = ["basnage","berault","merville","pesnelle","terrien"]
binpath = "/home/mpica/Progs/perso/CONDE/editions/base-version/"
einpath = "_base.xml"

listfile = "authors.csv"
tablefile = "mentions.csv"
checklist = "checklist.xml"

## FUNCTION: extract authors and store in a dictionary

In [78]:
authors = {}

def extract(witness, path):
    print(witness)
    
    """
    Function taking the name and path to a TEI-XML text file and
    analyzing the references, doing two things:
        - returning an XML element named after the current witness,
            itself containing the copy of each reference declaration,
        - completing the general author dictionnary with new elements,
            whether new authors or authors whose informations were
            incomplete.
            
    :param witness: Name (=id) of the current witness as str (no space)
    :param path: Path to the current witness TEI-XML file.
    
    """
    
    # Create the witness element.
    liste = ET.Element(witness)
    
    # Open and parse TEI-XML file.
    with open(path) as filein:
        tree = ET.parse(filein)
        root = tree.getroot()
        
        # Look for all declared authors.
        for author in tqdm(root.findall('.//{http://www.tei-c.org/ns/1.0}listPerson/{http://www.tei-c.org/ns/1.0}person')):
            
            # Get <birth> element.
            fullbirth = author.find('.//{http://www.tei-c.org/ns/1.0}birth')
            
            try:
                # Get current author identifier.
                ident = author.get("{http://www.w3.org/XML/1998/namespace}id")
                
                if ident not in authors.keys():
                    # Create a dict. entry for the current author.
                    authors[ident] = {}
            
                    try:
                        # Get current author birthdate. Add a ? marker if date unsure.
                        
                        if "when" in fullbirth.attrib.keys():
                            authors[ident]["birth"] = fullbirth.get("when")
                        else:
                            authors[ident]["birth"] = "?" + fullbirth.get("notBefore")
                            
                    except:
                        authors[ident]["birth"] = "none"
                        liste.append(author)

                    try:
                        # Create a dict. to store all recorded names for current author.
                        lg = {}
                        
                        # Loop on names, store their language.
                        for name in author.findall('.//{http://www.tei-c.org/ns/1.0}persName'):
                            namelang = name.get("{http://www.w3.org/XML/1998/namespace}lang")
                            
                            if name.text:
                                # If name is not split into <forename>/<surname> elements,
                                # there is text directly into <persName> element and we
                                # make this the current language text.
                                lg[namelang] = name.text
                                
                            else:
                                # If name is split, the order is unsure, therefore
                                # we store each kind into its own entry
                                # within names dict. and make a final str out of it.
                                names = {}
                                for nchild in name.findall('*'):
                                    if nchild.tag == "{http://www.tei-c.org/ns/1.0}forename":
                                        names["fn"] = nchild.text
                                    elif nchild.tag == "{http://www.tei-c.org/ns/1.0}surname":
                                        names["sn"] = nchild.text
                                    
                                lg[namelang] = names["sn"] + ", " + names["fn"]
                                
                        
                        # Setting an order of preference for final display of name:
                        # preferably French, if not, Latin, and if neither, English.
                        # (These are the only three name languages within the corpus.)
                        if "fr" in lg.keys():
                            authors[ident]["name"] = lg["fr"]
                        elif "la" in lg.keys():
                            authors[ident]["name"] = lg["la"]
                        elif "eng" in lg.keys():
                            authors[ident]["name"] = lg["eng"]
                            
                    except:
                        authors[ident]["name"] = "none"
                        liste.append(author)
                    
                # If the author was recorded in a previous witness but has no name,
                # we try to make a name string again with this witness.
                
                elif authors[ident]["name"] == "none":
                    
                    try:
                        lg = {}
                        for name in author.findall('.//{http://www.tei-c.org/ns/1.0}persName'):
                            namelang = name.get("{http://www.w3.org/XML/1998/namespace}lang")
                            if name.text:
                                lg[namelang] = name.text
                                
                            else:
                                names = {}
                                for nchild in name.findall('*'):
                                    if nchild.tag == "{http://www.tei-c.org/ns/1.0}forename":
                                        names["fn"] = nchild.text
                                    elif nchild.tag == "{http://www.tei-c.org/ns/1.0}surname":
                                        names["sn"] = nchild.text
                                        
                                lg[namelang] = names["sn"] + ", " + names["fn"]
                                                            
                        if "fr" in lg.keys():
                            authors[ident]["name"] = lg["fr"]
                        elif "la" in lg.keys():
                            authors[ident]["name"] = lg["la"]
                        elif "eng" in lg.keys():
                            authors[ident]["name"] = lg["eng"]
                            
                    except:
                        authors[ident]["name"] = "none"
                        liste.append(author)
                    
            except:
                continue
    
    return liste

## FUNCTION : sort extracted authors: keep those born before year 550

In [82]:
def sort(dico):
    
    """
    Function taking a dictionary of authors shaped like so:
    {'authorID': {'birth':'0000', 'name':'AuthorName'}}
    and returning the same dictionnary where all authors with a
    number greater than 550 as a birthdate were removed.
    
    :param dico: dict
    
    """
    print("Now sorting authors.")
    # Looping on author identifiers (=keys of dict.)
    for author in tqdm(dico.keys()):
    
        # If a ? sign is found, it is removed from the local string
        # but untouched in the dictionary.
        
        if "?" in authors[author]["birth"]:
            birth = authors[author]["birth"].replace("?","")
        else:
            birth = authors[author]["birth"]
        
        try:
            # If the author was born before 550, the entry is added
            # to the new dictionary.
            
            if int(birth) < 550:
                final[author] = authors[author]
        except:
            
            # If there is an error (no birth date), print the author
            # as we want to know if they are interesting now
            # (if so, we can correct the XML itself).
            
            print(authors[author])
    
    # Show me the final dictionary to assess the data.
    print(final)
    
    return final

## Using the previously declared functions

In [83]:
listroot = ET.Element("people")

for witness in witnesses:
    fullpath = binpath + witness + einpath
    
    listroot.append(extract(witness, fullpath))

final = {}

final = sort(authors)

with open(checklist, "w") as failures:
    a_ecrire = ET.tostring(listroot, encoding="unicode", method="xml")
    failures.write(a_ecrire)

basnage


  0%|          | 0/360 [00:00<?, ?it/s]

berault


  0%|          | 0/145 [00:00<?, ?it/s]

merville


...

terrien


  0%|          | 0/92 [00:00<?, ?it/s]

Now sorting authors.


  0%|          | 0/489 [00:00<?, ?it/s]

{'birth': 'none', 'name': 'Anian'}
{'birth': 'none', 'name': 'Herold, Johann Berthold'}
{'birth': 'none', 'name': 'Arq.'}
{'birth': 'none', 'name': 'Goncanus'}
{'birth': 'none', 'name': 'Febur.'}
{'birth': 'none', 'name': '\u2028Michel (saint)\u2028'}
{'birth': 'none', 'name': 'Masurier'}
{'birth': 'none', 'name': 'Chartier, Alexis'}
{'birth': 'none', 'name': 'Apulée'}
{'birth': 'none', 'name': 'Ald. Manut.'}
{'birth': 'none', 'name': 'Frigentius'}
{'birth': 'none', 'name': 'Fillesac'}
{'birth': 'none', 'name': 'Isidore'}
{'birth': 'none', 'name': 'Adrien (pape) '}
{'birth': 'none', 'name': '\u2028Matthieu (saint)\u2028'}
{'birth': 'none', 'name': 'Fortin'}
{'birth': 'none', 'name': 'Adrien'}
{'birth': 'none', 'name': 'Antipater de Thessalonique'}
{'birth': 'none', 'name': 'Moïse'}
{'birth': 'none', 'name': 'Froben'}
{'birth': 'none', 'name': '\u2028Thomas (saint)\u2028'}
{'birth': 'none', 'name': 'Josephe'}
{'birth': 'none', 'name': 'Balduin'}
{'birth': 'none', 'name': 'Jean-Baptiste 